In [ ]:
import os

## =====================================================================================
## This is a temporarly fix for the freezing and the cuda issues. You can add this
## utility script instead of kaggle_l5kit until Kaggle resolve these issues.
## 
## You will be able to train and submit your results, but not all the functionality of
## l5kit will work properly.

## More details here:
## https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/177125

## this script transports l5kit and dependencies
os.system('pip install --target=/kaggle/working pymap3d==2.1.0')
os.system('pip install --target=/kaggle/working protobuf==3.12.2')
os.system('pip install --target=/kaggle/working transforms3d')
os.system('pip install --target=/kaggle/working zarr')
os.system('pip install --target=/kaggle/working ptable')

os.system('pip install --no-dependencies --target=/kaggle/working l5kit')
os.system('pip install timm')

In [ ]:
# import packages
import os, gc
import zarr
import numpy as np 
import pandas as pd 
import timm
from tqdm import tqdm
from typing import Dict
from collections import Counter
from prettytable import PrettyTable
from collections import OrderedDict
import math
import pickle

#level5 toolkit
from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.data import ChunkedDataset, LocalDataManager

# level5 toolkit 
from l5kit.configs import load_config_data
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from l5kit.visualization import draw_trajectory, draw_reference_trajectory, TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, write_gif
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset, export_zarr_to_csv, write_gt_csv
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from colorama import Fore, Back, Style

# deep learning
import torch
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18, resnet50, resnet34
from torchvision.models.mobilenet import mobilenet_v2
from torch.nn import functional as F
# check files in directory
print((os.listdir('../input/lyft-motion-prediction-autonomous-vehicles/')))

plt.rc('animation', html='jshtml')

%matplotlib inline

# Validate Configuration

In [ ]:
DEBUG = False

# training cfg
training_cfg = {
    
    'format_version': 4,
    
     ## Model options
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    ## Input raster parameters
    'raster_params': {
        
        'raster_size': [300, 300], # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
        'pixel_size': [0.5, 0.5], # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
        'ego_center': [0.25, 0.5],
        'map_type': "py_semantic",
        
        # the keys are relative to the dataset environment variable
        'satellite_map_key': "aerial_map/aerial_map.png",
        'semantic_map_key': "semantic_map/semantic_map.pb",
        'dataset_meta_key': "meta.json",

        # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
        # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },

    ## Data loader options
    'train_data_loader': {
        'key': "scenes/train.zarr",
        'batch_size': 12,
        'shuffle': True,
        'num_workers': 4
    },

    ## Train params
    'train_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 100 if DEBUG else 12500
    }
}

# inference cfg
inference_cfg = {
    
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
        'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 4
    }

}

In [ ]:
# validate cfg
validate_cfg = {
    
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },
    
    'validate_data_loader': {
    'key': 'scenes/validate.zarr',
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 4
    }

}

# ***Model Definition***

# LyftModel——resnet50

In [ ]:
# class LyftModel(nn.Module):
    
#     def __init__(self, cfg):
#         super().__init__()
        
#         # set pretrained=True while training
#         self.backbone = resnet50(pretrained=True) 
        
#         num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
#         num_in_channels = 3 + num_history_channels

#         self.backbone.conv1 = nn.Conv2d(
#             num_in_channels,
#             self.backbone.conv1.out_channels,
#             kernel_size=self.backbone.conv1.kernel_size,
#             stride=self.backbone.conv1.stride,
#             padding=self.backbone.conv1.padding,
#             bias=False,
#         )
        
#         # This is 512 for resnet18 and resnet34;
#         # And it is 2048 for the other resnets
#         backbone_out_features = 2048
        
#         # X, Y coords for the future positions (output shape: Bx50x2)
#         num_targets = 2 * cfg["model_params"]["future_num_frames"]

#         # You can add more layers here.
#         self.head = nn.Sequential(
#             # nn.Dropout(0.2),
#             nn.Linear(in_features=backbone_out_features, out_features=4096),
#         )

#         self.logit = nn.Linear(4096, out_features=num_targets)
        
#     def forward(self, x):
#         x = self.backbone.conv1(x)
#         x = self.backbone.bn1(x)
#         x = self.backbone.relu(x)
#         x = self.backbone.maxpool(x)

#         x = self.backbone.layer1(x)
#         x = self.backbone.layer2(x)
#         x = self.backbone.layer3(x)
#         x = self.backbone.layer4(x)

#         x = self.backbone.avgpool(x)
#         x = torch.flatten(x, 1)
        
#         x = self.head(x)
#         x = self.logit(x)
        
#         return x

# mixnet_xl

In [ ]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        
        # set pretrained=True while training
        self.backbone = timm.create_model('mixnet_xl',pretrained=True)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv_stem = nn.Conv2d(
            num_in_channels,
            self.backbone.conv_stem.out_channels,
            kernel_size=self.backbone.conv_stem.kernel_size,
            stride=self.backbone.conv_stem.stride,
            padding=self.backbone.conv_stem.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 1536
        
        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone.conv_stem(x)
        x = self.backbone.bn1(x)
        x = self.backbone.act1(x)
        
        x = self.backbone.blocks(x)

        x = self.backbone.conv_head(x)
        x = self.backbone.bn2(x)
        x = self.backbone.act2(x)
        x = self.backbone.global_pool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x

# Hyperparameters

In [ ]:
# compiling model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftModel(training_cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# criterion = nn.MSELoss(reduction="none")
criterion = nn.SmoothL1Loss(reduction="none")

In [ ]:
device

In [ ]:
# root directory
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

#submission
SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

# Load data

In [ ]:
# validation configuration
valid_cfg = validate_cfg["validate_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(validate_cfg, dm)

# Validation dataset/dataloader
valid_zarr = ChunkedDataset(dm.require(valid_cfg["key"])).open()
valid_dataset = AgentDataset(validate_cfg, valid_zarr, rasterizer)
whole_size = valid_dataset.__len__()
valid_dataset_use, valid_dataset_valid, _ = torch.utils.data.random_split(valid_dataset, [70000, 5000, whole_size-75000], generator=torch.Generator().manual_seed(42))

valid_dataloader = DataLoader(valid_dataset_use,
                             shuffle=valid_cfg["shuffle"],
                             batch_size=valid_cfg["batch_size"],
                             num_workers=valid_cfg["num_workers"])

valid_dataloader_valid = DataLoader(valid_dataset_valid,
                             shuffle=valid_cfg["shuffle"],
                             batch_size=valid_cfg["batch_size"],
                             num_workers=valid_cfg["num_workers"])

print(valid_dataloader.dataset.__len__())


In [ ]:
valid_dataset_use, valid_dataset_valid, _ = torch.utils.data.random_split(valid_dataset, [70000, 5000, whole_size-75000], generator=torch.Generator().manual_seed(42))

valid_dataloader = DataLoader(valid_dataset_use,
                             shuffle=valid_cfg["shuffle"],
                             batch_size=valid_cfg["batch_size"],
                             num_workers=valid_cfg["num_workers"])

valid_dataloader_valid = DataLoader(valid_dataset_valid,
                             shuffle=valid_cfg["shuffle"],
                             batch_size=valid_cfg["batch_size"],
                             num_workers=valid_cfg["num_workers"])

print(valid_dataloader.dataset.__len__())

# Load previous pth

In [ ]:
WEIGHT_FILE = '../input/4th-train/model_state_mixnet_xl_12000.pth'
model_state = torch.load(WEIGHT_FILE, map_location=device)
model.load_state_dict(model_state)

# Final Evaluation

In [ ]:
# Final - Evaluate Validation Dataset 
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []
# coordinates ground truth
valid_coords_gts = []
# target avalabilities
target_avail_pd = []
agent_ids = []
progress_bar = tqdm(valid_dataloader)
for data in progress_bar:
    
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    
    outputs = model(inputs).reshape(targets.shape)
    
    future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())
    valid_coords_gts.append(data["target_positions"].numpy().copy())
    target_avail_pd.append(target_availabilities.cpu().numpy().copy())

# Concatenate Data

In [ ]:
timestamps_concat = np.concatenate(timestamps)
track_ids_concat = np.concatenate(agent_ids)
coords_concat = np.concatenate(future_coords_offsets_pd)
gt_valid_final = np.concatenate(valid_coords_gts)
target_avail_concat = np.concatenate(target_avail_pd)


In [ ]:
# submission.csv
write_gt_csv(
    csv_path="mixnet_xl_12500_valid_gt_rand.csv", 
    timestamps=timestamps_concat, 
    track_ids=track_ids_concat, 
    coords=gt_valid_final, 
    avails=target_avail_concat.squeeze(-1)
)

write_pred_csv('submission_mixnet_xl_12500_val.csv',
               timestamps=timestamps_concat,
               track_ids=track_ids_concat,
               coords=coords_concat,
              )

In [ ]:
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'submission_mixnet_xl_12500_val.csv')

In [ ]:
FileLink(r'mixnet_xl_12500_valid_gt_rand.csv')

# Compute metrics

In [ ]:
# Negative Log Likelihood Metrics
eval_gt_path = "./mixnet_xl_12500_valid_gt_rand.csv"
pred_path = "./submission_mixnet_xl_12500_val.csv"

metrics = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, time_displace])
for metric_name, metric_mean in metrics.items():
    print(metric_name, metric_mean)

# Plot Prediction Tractories

In [ ]:
model.eval()
torch.set_grad_enabled(False)

# Uncomment to choose satelliter or semantic rasterizer
# validate_cfg["raster_params"]["map_type"] = "py_satellite"
validate_cfg["raster_params"]["map_type"] = "py_semantic"


rast = build_rasterizer(validate_cfg, dm)

eval_ego_dataset = EgoDataset(validate_cfg, valid_dataset.dataset, rast)
num_frames = 10 # randomly pick _ frames
random_frames = np.random.randint(0,len(eval_ego_dataset)-1, (num_frames,))

for frame_number in random_frames:  
    agent_indices = valid_dataset.get_frame_indices(frame_number) 
    if not len(agent_indices):
        continue

    # get AV point-of-view frame
    data_ego = eval_ego_dataset[frame_number]
    im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
    center = np.asarray(validate_cfg["raster_params"]["ego_center"]) * validate_cfg["raster_params"]["raster_size"]
    
    predicted_positions = []
    target_positions = []

    for v_index in agent_indices:
        data_agent = valid_dataset[v_index]

        out_net = model(torch.from_numpy(data_agent["image"]).unsqueeze(0).to(device))
        out_pos = out_net[0].reshape(-1, 2).detach().cpu().numpy()
        # store absolute world coordinates
        predicted_positions.append(transform_points(out_pos, data_agent["world_from_agent"]))
        # retrieve target positions from the GT and store as absolute coordinates
        track_id, timestamp = data_agent["track_id"], data_agent["timestamp"]
        target_positions.append(transform_points(data_agent["target_positions"], data_agent["world_from_agent"]) )

    # convert coordinates to AV point-of-view so we can draw them
    predicted_positions = transform_points(np.concatenate(predicted_positions), data_ego["raster_from_world"])
    target_positions = transform_points(np.concatenate(target_positions), data_ego["raster_from_world"])
    
    # make sure ground truth and prediction have the same data size
    assert len(target_positions) == len(predicted_positions)
    
    # draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
    
    plt.rcParams['figure.figsize'] = 6, 6
    plt.imshow(im_ego[::-1])
#     plt.show()
    
    draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    
    plt.rcParams['figure.figsize'] = 6, 6
    plt.imshow(im_ego[::-1])
    plt.show()